# Face recognizer using pre trained model

Below is the implementation of a face recognizer which can identify the face of the person showing on a web cam. We will apply transfer learning and use pre-trained weights of VGG Face model.

# Import the requried libraries
Here we are importing all the requried libraries.

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
import cv2
import os

# Contructing the neural network model
The model here constructed is based on VGG face model.It has 22 layers and 37 deep units.
![Vgg face model](images/vgg-face-model.png)
The structure of the VGG-Face model is demonstrated below.


The VGG-Face architure can be to be understood clear with the help of following image,
![Vgg face model](images/vgg-face-architecture.jpg)



In [ ]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

# Loading the model with pretrained weights



In [ ]:



from keras.models import model_from_json

model.load_weights('C:\\Users\\Dell\\Desktop\\mlops\\Face-recognizer-using-pre-trained-model-master\\Face-recognizer-using-pre-trained-model-masterf/vgg_face_weights.h5')
# model.load_weights('C:/Users/Abhay/Desktop/Face-recognition-using-deep-learning-master/vgg_face_weights.h5')

## About <font color=blue>image_to_embedding</font> function        
When the model is loaded with pre trained weights, then we can create the **2622 dimensional embedding vectors** for all the face images stored in the "images" folder. **"image_to_embedding"** function pass an image to the VGG network to generate the embedding vector.

In [ ]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (224, 224)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


## About <font color=blue>recognize_face</font> function
This function calculate similarity between the captured image and the images that are already been stored in the folder. It passes the image to the trained neural network to generate its embedding vector. Which is then compared with all the embedding vectors of the images stored by calculating L2 Euclidean distance and the cosine distance. 

Here I am using both the [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity) and the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) to see for the match.Greater the Cosine similarity between two images then there is greater probability of having the image of same person. Smaller the Euclidean distance between two images then there is greater probability of having the image of same person. 

Here I am setting the thresholds for the Euclidean distance and Cosine Similarity are 0.17 and 0.70 (which can be adjusted).

In [ ]:
def recognize_face(face_image, input_embeddings, model):   
    
    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    minimum_distance1 = 0
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding) * 1000
        
        a = np.matmul(embedding,input_embedding.T)
        b = np.sqrt(np.matmul(embedding,embedding.T))
        c = np.sqrt(np.matmul(input_embedding,input_embedding.T))
        cosine_dist = a / ( b * c )
        
        #print(dist1)
        #print (cosine_dist)
        
      
        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance and cosine_dist > minimum_distance1:
            minimum_distance = euclidean_distance
            minimum_distance1 = cosine_dist
            name = input_name[0]
            
    
    if minimum_distance < 0.17 and minimum_distance1 > 0.70:
        return str(name)
    else:
        return None
    

## About <font color=blue>create_input_image_embeddings</font> function
This function generates 512 dimensional image ebeddings of all the images stored in the "imagesv1" containg folders with person  names with images.We feed forwarding the images to a trained neural network. It creates a dictionary with key as the name of the face and value as embedding


In [ ]:
import glob

def create_input_image_embeddings():      
    input_embeddings = {}

    for file in glob.glob("imagesv1/*"):
        for file1 in glob.glob(file+"/*"):
            person_name_image = os.path.splitext(os.path.basename(file1))[0]
            person_name = os.path.splitext(os.path.basename(file))[0]
#             print(person_name)
            image_file = cv2.imread(file1, 1)
            input_embeddings[person_name,person_name_image] = image_to_embedding(image_file, model)

    return input_embeddings

## About <font color=blue>recognize_faces_in_cam</font> function
This function capture image from the webcam, detect a face in it and crop the image to have a face only, which is then passed to recognize_face function. Here for the face detection we are using Haarcascade model.

In [ ]:
def recognize_faces_in_cam(input_embeddings):
    

    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            

            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Capturing the face image
Following code captures 10 face images of the person. They all are stored in **"imagesv1"** folder.After every run this will create a new folder with name **"new user_(current date and time)"** (inorder to avoid duplication of the folder name).In this sub folder there will be 10 images of detected faces will be strored.You have to rename the folder name with the person name in order to get the correct name of person in output.You can also remove the images which are not much good from set of 10 images.

These images will be used for recognizing the the identity of the person using one shot learning.

In [ ]:
cam = cv2.VideoCapture(0)

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#  #change - to get current time and date
from datetime import datetime
date_time = datetime.now().strftime('%Y%m%d%H%M%S')

#  #change create a new folder to save images
newpath =r'path'+date_time 
image_path = r'imagesv1\newUser'+date_time
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
    
count = 0
while(True):
    ret, img = cam.read()
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite(image_path+"/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 10: # Take 10 face sample and stop video
         break
cam.release()
cv2.destroyAllWindows()

In [ ]:
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)
